In [38]:
import numpy as np
import pandas as pd
import pandas_ta as Ta
import plotly.graph_objects as go
import torch
import matplotlib.pyplot as plt
import datetime
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [39]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [40]:
dataset = yf.download(tickers="AAPL", start = "2013-01-01", end = "2023-12-31")
dataset

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2013-01-02,19.779285,19.821428,19.343929,19.608213,16.705700,560518000
2013-01-03,19.567142,19.631071,19.321428,19.360714,16.494843,352965200
2013-01-04,19.177500,19.236786,18.779642,18.821428,16.035379,594333600
2013-01-07,18.642857,18.903570,18.400000,18.710714,15.941063,484156400
2013-01-08,18.900356,18.996071,18.616072,18.761070,15.983955,458707200
...,...,...,...,...,...,...
2023-12-22,195.179993,195.410004,192.970001,193.600006,192.868134,37122800
2023-12-26,193.610001,193.889999,192.830002,193.050003,192.320221,28919300
2023-12-27,192.490005,193.500000,191.089996,193.149994,192.419830,48087700


In [42]:
dataset['RSI_14'] = Ta.rsi(dataset["Adj Close"])
dataset['SMA_50'] = Ta.sma(dataset["Adj Close"], length=50)  # 50-period simple moving average
dataset['SMA_200'] = Ta.sma(dataset["Adj Close"], length=200)  # 200-period simple moving average
dataset['EMA_12'] = Ta.ema(dataset["Adj Close"], length=12)  # 12-period exponential moving average
dataset['EMA_26'] = Ta.ema(dataset["Adj Close"], length=26)  # 26-period exponential moving average
dataset['ATR'] = Ta.atr(dataset['High'], dataset['Low'], dataset["Adj Close"])
dataset = dataset.dropna()


C:\Users\khiettam\AppData\Local\Temp\ipykernel_21180\1692209771.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\khiettam\AppData\Local\Temp\ipykernel_21180\1692209771.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\khiettam\AppData\Local\Temp\ipykernel_21180\1692209771.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

In [43]:
dataset

,Open,High,Low,Close,Adj Close,Volume,RSI_14,SMA_50,SMA_200,EMA_12,EMA_26,ATR
Date,,,,,,,,,,,,
2014-08-01,23.725000,24.155001,23.702499,24.032499,21.233017,194044000,54.520052,20.602175,17.858872,21.306664,21.022653,2.977927
2014-08-04,24.092501,24.145000,23.792500,23.897499,21.113737,159832000,52.103057,20.641214,17.886741,21.276983,21.029400,2.973217
2014-08-05,23.840000,23.920000,23.590000,23.780001,21.009932,223732000,50.024600,20.673847,17.913565,21.235898,21.027958,2.961292
2014-08-06,23.687500,23.870001,23.677500,23.740000,20.974592,154232000,49.303527,20.698516,17.939532,21.195697,21.024005,2.954062
2014-08-07,23.732500,23.987499,23.525000,23.620001,20.972366,186844000,49.255369,20.724163,17.963555,21.161339,21.020180,2.958265
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22,195.179993,195.410004,192.970001,193.600006,192.868134,37122800,54.672794,184.605997,177.602992,194.001164,191.353204,2.894590
2023-12-26,193.610001,193.889999,192.830002,193.050003,192.320221,28919300,53.090055,184.893612,177.827889,193.742557,191.424835,2.763547
2023-12-27,192.490005,193.500000,191.089996,193.149994,192.419830,48087700,53.354431,185.185806,178.043510,193.539061,191.498538,2.738294


In [32]:
# Create a candlestick chart
fig = go.Figure(data=[go.Candlestick(x=dataset.index,
                                     open=dataset['Open'],
                                     high=dataset['High'],
                                     low=dataset['Low'],
                                     close=dataset['Close'])])

fig.add_trace(go.Scatter(x=dataset.index, y=dataset['SMA_50'], mode='lines', name='SMA 50'))
fig.add_trace(go.Scatter(x=dataset.index, y=dataset['RSI_14'], mode='lines', name='RSI_14'))
fig.add_trace(go.Scatter(x=dataset.index, y=dataset['ATR'], mode='lines', name='ATR'))

# Add titles and labels
fig.update_layout(title="AAPL Stock Price - Candlestick Chart",
                  xaxis_title="Date",
                  yaxis_title="Price (USD)",
                  xaxis_rangeslider_visible=False)

# Show the plot
fig.show()

In [17]:
scaler = MinMaxScaler()
dataset = pd.DataFrame(scaler.fit_transform(dataset))

In [18]:
dataset

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.032163,0.029944,0.030507,0.030738,0.025638,0.373377,NaN,NaN,NaN,NaN,NaN,NaN
1,0.031011,0.028917,0.030385,0.029394,0.024501,0.228923,NaN,NaN,NaN,NaN,NaN,NaN
2,0.028895,0.026789,0.027428,0.026465,0.022023,0.396912,NaN,NaN,NaN,NaN,NaN,NaN
3,0.025992,0.024992,0.025356,0.025864,0.021514,0.320230,NaN,NaN,NaN,NaN,NaN,NaN
4,0.027390,0.025491,0.026535,0.026138,0.021745,0.302518,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2763,0.984579,0.977286,0.978008,0.975511,0.975767,0.009100,0.484349,0.990547,0.994845,0.998441,0.998543,0.180635
2764,0.976054,0.969085,0.977244,0.972524,0.972812,0.003390,0.461693,0.992209,0.996212,0.997018,0.998944,0.152254
2765,0.969972,0.966981,0.967748,0.973067,0.973349,0.016731,0.465477,0.993898,0.997523,0.995898,0.999356,0.146785
2766,0.978932,0.973240,0.979099,0.975402,0.975660,0.006961,0.482462,0.995817,0.998784,0.995313,0.999916,0.139079
